## Outils pour la manipulation d'images et librairies.


In [287]:
import PIL
from PIL import Image
import numpy as np
#import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

Question 1 : Donner le code qui transforme une image RGB en une image YCbCr. Vous pourrez produire
une matrice pour chaque composante, pour pouvoir plus facilement les manipuler indépendamment. Vous
pouvez stocker les données YCbCr comme des entiers ou des flottants, mais vous expliquerez votre choix.

In [288]:
def RGB_en_YCbCR(image):
    image_finale = np.array(image, dtype='float64')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R = image[i, j, 0]
            G = image[i, j, 1]
            B = image[i, j, 2]
            image_finale[i, j, 0] = 0.299 * R + 0.587 * G + 0.114 * B
            image_finale[i, j, 1] = -0.1687 * R - 0.3313 * G + 0.5 * B + 128
            image_finale[i, j, 2] = 0.5 * R - 0.4187 * G - 0.0813 * B + 128
    return image_finale


Question 2 : Donner le code qui transforme une image YCbCr en une image RGB. Attention, les valeurs des
canaux RGB doivent être un entier dans [0, 255] qui pourra être codé sur un octet. Appliquer successivement
la transformation RGB vers YCbCr puis YCbCr vers RGB et vérifier que vous obtenez l’image de départ.
Vous consulterez la documentation des fonctions de numpy clip, uint8 et mask qui pourraient vous être
utiles.

In [289]:
def YCbCR_en_RGB(image):
    image_finale = np.array(image, dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Y = image[i, j, 0]
            Cb = image[i, j, 1]
            Cr = image[i, j, 2]
            image_finale[i, j, 0] = Y + 1.402 * (Cr - 128)
            image_finale[i, j, 1] = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
            image_finale[i, j, 2] = Y + 1.772 * (Cb - 128)
    return np.uint8(np.clip(image_finale, 0, 255))


Question 3 : Donner la fonction qui réalise ce padding ainsi que celle qui l’élimine et vérifier que
l’application de ces deux transformations laissent l’image inchangée.

In [290]:

def padding(image):
    padded_image = np.array(image, dtype='uint8')
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]
    canaux = padded_image.shape[2]
    if ligne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne%8, colonne, canaux), dtype=np.uint8)), axis=0)
    if colonne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne, colonne%8, canaux), dtype=np.uint8)), axis=1)
    return padded_image

In [291]:
def unpadding(padded_image, image_origine):
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]

    ligne_origine = image_origine.shape[0]
    colonne_origine = image_origine.shape[1]

    # Calculer le nombre de colonnes à enlever
    nb_col_enlever = colonne - colonne_origine
    while nb_col_enlever > 0:
        padded_image = padded_image[:, :-1]
        nb_col_enlever -= 1

    # Calculer le nombre de lignes à enlever
    nb_ligne_enlever = ligne- ligne_origine
    while nb_ligne_enlever > 0:
        padded_image = padded_image[:-1, :]
        nb_ligne_enlever -= 1

    return padded_image

Question 4 : Implémenter la fonction qui sous-échantillonne une matrice et renvoie une matrice deux fois
plus petite.

In [292]:
def sous_echantillonage(matrice):
    
    new_len_col = len(matrice[1]) // 2

    # Sous-échantillonner la matrice
    matrice_se = [[i for i in range(new_len_col)] for i in range(len(matrice))]
    for i in range(len(matrice)):
        for j in range(0,len(matrice[1]),2):
            new_value = (matrice[i][j] + matrice[i][j+1])/2
            matrice_se[i][j//2] = new_value

    return matrice_se

## Découpage en blocs et compression

## Écriture dans un fichier

## Décompression

## Tests 

In [293]:
test = load("test.png")
#testYCbCr = YCbCR_en_RGB(RGB_en_YCbCR(test))
#Image.fromarray(test,'RGB').show()
#Image.fromarray(testYCbCr,'RGB').show()

#Image.fromarray(padding(test),'RGB').show()
#Image.fromarray(unpadding(padding(test), test),'RGB').show()

print(sous_echantillonage([[1,2,3,6],[3,5,10,0]]))

[[1.5, 4.5], [4.0, 5.0]]
